# Volatility Surfaces from Heston vs Black–Scholes
This notebook is generated from `IV.pynb` (originally a Python script) and reproduces the same logic inside executable cells.

In [1]:
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from scipy.stats import norm
import plotly.io as pio # Import plotly.io to save the plot

# Data for the volatility surface - precompute meshgrid
strikes = [90, 95, 100, 105, 110]
maturities = [1/12, 3/12, 6/12, 1, 2]    # in years
maturity_labels = ['1 Month', '3 Months', '6 Months', '1 Year', '2 Years']

# Market volatility values (in %)
market_vols = np.array([
    [28.0, 24.5, 22.0, 20.5, 19.5],  # 1 month
    [27.5, 24.0, 21.8, 20.3, 19.3],  # 3 months
    [27.0, 23.5, 21.5, 20.0, 19.0],  # 6 months
    [26.5, 23.0, 21.2, 19.8, 18.8],  # 1 year
    [26.0, 22.5, 21.0, 19.5, 18.5]   # 2 years
])

# Create meshgrid for 3D surface
X, Y = np.meshgrid(strikes, maturities)

def heston_paths(S0, v0, kappa, theta, xi, rho, r, T, N, M):
    """
    Simulate Heston paths using Euler discretization
    S0: initial stock price
    v0: initial variance
    kappa: mean reversion speed
    theta: long-run variance
    xi: volatility of variance
    rho: correlation
    r: risk-free rate
    T: time horizon
    N: number of time steps
    M: number of paths
    """
    dt = T/N
    
    # Initialize arrays
    S = np.zeros((N+1, M))
    v = np.zeros((N+1, M))
    S[0] = S0
    v[0] = v0
    
    # Generate correlated random numbers
    Z1 = np.random.standard_normal((N, M))
    Z2 = rho * Z1 + np.sqrt(1-rho**2) * np.random.standard_normal((N, M))
    
    # Simulate paths
    for i in range(N):
        # Ensure variance remains non-negative
        v[i+1] = np.maximum(v[i] + kappa*(theta - v[i])*dt + xi*np.sqrt(v[i]*dt)*Z1[i], 0)
        S[i+1] = S[i] * np.exp((r - 0.5*v[i])*dt + np.sqrt(v[i]*dt)*Z2[i])
    
    return S, v

def bs_implied_vol(S0, K, T, r, price, call=True):
    """Calculate BS implied volatility using Newton-Raphson"""
    def bs_price(sigma):
        # Black-Scholes price formula
        d1 = (np.log(S0/K) + (r + 0.5*sigma**2)*T)/(sigma*np.sqrt(T))
        d2 = d1 - sigma*np.sqrt(T)
        if call:
            return S0*norm.cdf(d1) - K*np.exp(-r*T)*norm.cdf(d2)
        else:
            return K*np.exp(-r*T)*norm.cdf(-d2) - S0*norm.cdf(-d1)
    
    sigma = 0.3  # Initial guess for volatility
    for _ in range(100): # Max 100 iterations for convergence
        price_diff = bs_price(sigma) - price
        if abs(price_diff) < 1e-5: # Convergence criterion
            return sigma
        d1 = (np.log(S0/K) + (r + 0.5*sigma**2)*T)/(sigma*np.sqrt(T))
        vega = S0*np.sqrt(T)*norm.pdf(d1) # Vega for Newton-Raphson
        if vega == 0: # Avoid division by zero
            return sigma
        sigma = sigma - price_diff/vega
        if sigma <= 0: # Ensure sigma remains positive
            sigma = 0.01
    return sigma # Return the best estimate if not converged

# Fixed parameters for plot
S0 = 100
v0 = 5.208/100  # Initial variance (converted from percentage)
kappa = 2.0
theta = 0.04
xi = 0.3
rho = -0.7
r = 0.02

# Calculate Heston implied volatility surface
heston_vols = np.zeros_like(market_vols)
N_paths = 100000 # Number of simulation paths

for i, T in enumerate(maturities):
    N_steps = int(T * 252)  # Daily steps (assuming 252 trading days in a year)
    if N_steps == 0: # Ensure at least one step for very short maturities
        N_steps = 1
    for j, K in enumerate(strikes):
        # Simulate Heston paths
        S, v = heston_paths(S0, v0, kappa, theta, xi, rho, r, T, N_steps, N_paths)
        
        # Calculate option price (for a call option)
        payoffs = np.maximum(S[-1] - K, 0)
        price = np.exp(-r*T) * np.mean(payoffs) # MC call price approx
        
        # Convert to implied vol
        try:
            impl_vol = bs_implied_vol(S0, K, T, r, price) * 100  # Convert to percentage
            heston_vols[i,j] = impl_vol
        except Exception as e:
            print(f"Could not calculate implied volatility for T={T}, K={K}: {e}")
            heston_vols[i,j] = np.nan # Assign NaN if calculation fails
# Create figure with two subplots
fig = make_subplots(
    rows=1, cols=2,
    specs=[[{'type': 'surface'}, {'type': 'surface'}]],
    subplot_titles=('Heston Implied Volatility Surface', 'BS Implied Volatility Surface')
)

# Add Heston surface
fig.add_trace(
    go.Surface(x=X, y=Y, z=heston_vols, colorscale='Viridis', opacity=0.7, showscale=True),
    row=1, col=1
)

# Add Market surface
fig.add_trace(
    go.Surface(x=X, y=Y, z=market_vols, colorscale='Viridis', opacity=0.7, showscale=True),
    row=1, col=2
)

# Update layout
fig.update_layout(
    title='Heston vs Market Implied Volatility Surfaces',
    scene=dict(
        xaxis_title='Strike Price',
        yaxis_title='Time to Maturity (Years)', 
        zaxis_title='Implied Volatility (%)',
        xaxis=dict(gridcolor='darkgray', showgrid=True, color='darkgray', backgroundcolor='rgb(30, 30, 35)'),
        yaxis=dict(gridcolor='darkgray', showgrid=True, color='darkgray', backgroundcolor='rgb(30, 30, 35)'),
        zaxis=dict(gridcolor='darkgray', showgrid=True, color='darkgray', backgroundcolor='rgb(30, 30, 35)'),
        bgcolor='rgba(0,0,0,0)',
        camera=dict(eye=dict(x=1.5, y=1.5, z=1.2))
    ),
    scene2=dict(
        xaxis_title='Strike Price',
        yaxis_title='Time to Maturity (Years)',
        zaxis_title='Implied Volatility (%)',
        xaxis=dict(gridcolor='darkgray', showgrid=True, color='darkgray', backgroundcolor='rgb(30, 30, 35)'),
        yaxis=dict(gridcolor='darkgray', showgrid=True, color='darkgray', backgroundcolor='rgb(30, 30, 35)'),
        zaxis=dict(gridcolor='darkgray', showgrid=True, color='darkgray', backgroundcolor='rgb(30, 30, 35)', range=[np.min(market_vols)-2, np.max(market_vols)+2]),
        bgcolor='rgba(0,0,0,0)',
        camera=dict(eye=dict(x=1.5, y=1.5, z=1.2))
    ),
    width=1000,
    height=600,
    paper_bgcolor='rgba(0,0,0,0)',
    font=dict(color='white')
)

fig.show()
